In [ ]:
# TODO: train Kalman, then try to predict next states/observations
# TODO: train Kalman and linear on one person, then run Kalman on other person and run through linear

In [ ]:
from matplotlib import pyplot as plt, animation as animation
import numpy as np

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

from pykalman import KalmanFilter

π = np.pi

# Sklearn softmax classification on hidden states of Kalman filter

In [ ]:
data_path = 'sc-agg-f16.npz'

data = np.load(data_path)

In [ ]:
data['labels']

array(['EEG FPZ-CZ-2.5hz', 'EEG FPZ-CZ-5.0hz', 'EEG FPZ-CZ-7.5hz',
       'EEG FPZ-CZ-10.0hz', 'EEG FPZ-CZ-12.5hz', 'EEG FPZ-CZ-15.0hz',
       'EEG FPZ-CZ-17.5hz', 'EEG FPZ-CZ-20.0hz', 'EEG FPZ-CZ-22.5hz',
       'EEG FPZ-CZ-25.0hz', 'EEG FPZ-CZ-27.5hz', 'EEG FPZ-CZ-30.0hz',
       'EEG FPZ-CZ-32.5hz', 'EEG FPZ-CZ-35.0hz', 'EEG FPZ-CZ-37.5hz',
       'EEG FPZ-CZ-40.0hz', 'EEG FPZ-CZ-42.5hz', 'EEG FPZ-CZ-45.0hz',
       'EEG FPZ-CZ-47.5hz', 'EEG FPZ-CZ-50.0hz', 'EEG PZ-OZ-2.5hz',
       'EEG PZ-OZ-5.0hz', 'EEG PZ-OZ-7.5hz', 'EEG PZ-OZ-10.0hz',
       'EEG PZ-OZ-12.5hz', 'EEG PZ-OZ-15.0hz', 'EEG PZ-OZ-17.5hz',
       'EEG PZ-OZ-20.0hz', 'EEG PZ-OZ-22.5hz', 'EEG PZ-OZ-25.0hz',
       'EEG PZ-OZ-27.5hz', 'EEG PZ-OZ-30.0hz', 'EEG PZ-OZ-32.5hz',
       'EEG PZ-OZ-35.0hz', 'EEG PZ-OZ-37.5hz', 'EEG PZ-OZ-40.0hz',
       'EEG PZ-OZ-42.5hz', 'EEG PZ-OZ-45.0hz', 'EEG PZ-OZ-47.5hz',
       'EEG PZ-OZ-50.0hz', 'EOG HORIZONTAL-2.5hz', 'EOG HORIZONTAL-5.0hz',
       'EOG HORIZONTAL-7.5hz', 'EOG HORI

In [ ]:
data['train_patients']

array(['SC4241', 'SC4242', 'SC4001', 'SC4002', 'SC4271', 'SC4272',
       'SC4761', 'SC4762', 'SC4021', 'SC4022', 'SC4341', 'SC4342',
       'SC4011', 'SC4012', 'SC4411', 'SC4412', 'SC4281', 'SC4282',
       'SC4431', 'SC4432', 'SC4041', 'SC4042', 'SC4081', 'SC4082',
       'SC4731', 'SC4732', 'SC4631', 'SC4632', 'SC4121', 'SC4122',
       'SC4051', 'SC4052', 'SC4061', 'SC4062', 'SC4031', 'SC4032',
       'SC4091', 'SC4092', 'SC4771', 'SC4772', 'SC4211', 'SC4212',
       'SC4451', 'SC4452', 'SC4251', 'SC4252', 'SC4111', 'SC4112',
       'SC4371', 'SC4372', 'SC4171', 'SC4172', 'SC4561', 'SC4562',
       'SC4471', 'SC4472', 'SC4581', 'SC4582', 'SC4741', 'SC4742',
       'SC4231', 'SC4232', 'SC4522', 'SC4751', 'SC4752', 'SC4651',
       'SC4652', 'SC4611', 'SC4612', 'SC4151', 'SC4152', 'SC4321',
       'SC4322', 'SC4701', 'SC4702'], dtype='<U6')

In [ ]:
patient = data['train_patients'][0]
print(patient)
X, y = data[patient][:, :-1], data[patient][:, -1]
T, N = X.shape
X.shape

SC4241


(2702, 64)

In [ ]:
sleep_stages=  ('m', 'w', '1', '2', '3', '4', 'r')
# dim_x doesn't necessarily need to be the same as num_stages
num_stages = len(sleep_stages)
dim_x = num_stages
dim_z = N

# Init
kf = KalmanFilter(n_dim_state=dim_x, n_dim_obs=dim_z)

# Fit to fourier data with expectation maximization
kf.em(X)

In [ ]:
# Predict the states for the fourier data
states, covs = kf.smooth(X)

In [ ]:
states.shape

(2702, 7)

In [ ]:
# Train-validation split
val_size = 0.3
train_states, val_states, train_labels, val_labels = train_test_split(states, y, test_size=val_size, random_state=42)

# Transform data to have mean zero and standard deviation one
# (otherwise, LogisticRegression says it doesn't converge though performs about the same)
scaler = StandardScaler().fit(train_states)
train_states = scaler.transform(train_states)
val_states = scaler.transform(val_states)

In [ ]:
model = LogisticRegression(max_iter=200)

model.fit(train_states, train_labels)

LogisticRegression(max_iter=200)

In [ ]:
print(f'train accuracy: {round(model.score(train_states, train_labels), 2)}')
print(f'val accuracy:   {round(model.score(val_states, val_labels), 2)}')

train accuracy: 0.93
val accuracy:   0.93


In [ ]:
train_pred = model.predict(train_states)

print('report')
print(classification_report(train_labels, train_pred, zero_division=np.nan))

print('proportions in classes')
print(np.unique(train_labels, return_counts=True)[1] / len(train_labels))
print()

print('confusion matrix')
print(np.round(confusion_matrix(train_labels, train_pred, normalize='true'), 2))

report
              precision    recall  f1-score   support

         0.0       0.99      0.99      0.99      1314
         1.0       0.10      0.02      0.03        59
         2.0       0.82      0.95      0.88       360
         3.0       0.78      0.26      0.39        27
         5.0       0.82      0.88      0.85       131

    accuracy                           0.93      1891
   macro avg       0.70      0.62      0.63      1891
weighted avg       0.91      0.93      0.92      1891

proportions in classes
[0.69487044 0.03120042 0.19037546 0.01427816 0.06927552]

confusion matrix
[[0.99 0.   0.   0.   0.  ]
 [0.17 0.02 0.68 0.   0.14]
 [0.01 0.   0.95 0.01 0.04]
 [0.   0.   0.74 0.26 0.  ]
 [0.   0.02 0.1  0.   0.88]]


# New data: Sklearn softmax classification on Fourier data

In [ ]:
# Train-validation split
val_size = 0.3
train_X, val_X, train_labels, val_labels = train_test_split(X, y, test_size=val_size, random_state=42)

# Transform data to have mean zero and standard deviation one
# (otherwise, LogisticRegression says it doesn't converge though performs about the same)
scaler = StandardScaler().fit(train_X)
train_states = scaler.transform(train_X)
val_states = scaler.transform(val_X)

In [ ]:
model = LogisticRegression()

model.fit(train_X, train_labels)

/home/schilln/.local/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [ ]:
print(f'train accuracy: {round(model.score(train_X, train_labels), 2)}')
print(f'val accuracy:   {round(model.score(val_X, val_labels), 2)}')

train accuracy: 0.94
val accuracy:   0.93


In [ ]:
train_pred = model.predict(train_X)

print('report')
print(classification_report(train_labels, train_pred, zero_division=np.nan))

print('proportions in classes')
print(np.unique(train_labels, return_counts=True)[1] / len(train_labels))
print()

print('confusion matrix')
print(np.round(confusion_matrix(train_labels, train_pred, normalize='true'), 2))

report
              precision    recall  f1-score   support

         0.0       0.99      0.99      0.99      1314
         1.0       0.58      0.25      0.35        59
         2.0       0.85      0.91      0.88       360
         3.0       0.70      0.52      0.60        27
         5.0       0.81      0.85      0.83       131

    accuracy                           0.94      1891
   macro avg       0.78      0.71      0.73      1891
weighted avg       0.93      0.94      0.93      1891

proportions in classes
[0.69487044 0.03120042 0.19037546 0.01427816 0.06927552]

confusion matrix
[[0.99 0.   0.   0.   0.  ]
 [0.1  0.25 0.51 0.   0.14]
 [0.01 0.02 0.91 0.02 0.04]
 [0.   0.   0.48 0.52 0.  ]
 [0.03 0.01 0.11 0.   0.85]]
